In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# landsat8 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}
# NDVI and mNDWI 函数的定义
def index(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(ndvi).addBands(mndwi)
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'


In [5]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(roi).select('constant').rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")
basemap1 = ee.Image.constant(0).clip(roi).select('constant').rename('water_class')

# 训练和分类

In [7]:
landsat8_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr)

landsat_ndvi_mndwi = landsat8_images.map(index)

In [ ]:
def imageSample(image):
    wuhan_mNDWI_NDVI = image.select('mNDWI').gt(image.select('NDVI'))
    wuhan_waterextent = wuhan_mNDWI_NDVI.clip(roi).select('mNDWI').rename('waterclass')
    # Make the training dataset.
    points = wuhan_waterextent.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 200,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    # Overlay the points on the imagery to get training.
    sample_point = image.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })
    return sample_point

In [ ]:
points_collection = landsat_ndvi_mndwi.map(imageSample).flatten()
print('total sample number:{}'.format(points_collection.size().getInfo()))
# print(points_collection.first().getInfo())
samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))
# print(samplepoint_water.first().getInfo())

trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
#对Landsat-8进行分类
def training(image):
    return image.clip(roi).select(bands).classify(trainedClassifier).eq(1).selfMask().select('classification').rename('waterclass')
landsatImage_classify = landsat_ndvi_mndwi.map(training)

In [8]:
# 计算水体像元
waterpixel_count = landsatImage_classify.sum()
# 计算有效像元
validPixel = landsat8_images.count().select('B3').clip(roi).rename('count')
# Map.addLayer(count_image,{},'count_image')

wuhan_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).select('waterclass').rename('frequency')
wuhan_permanentwater = wuhan_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')

Map.addLayer(wuhan_permanentwater,{'palette':['yellow']},"water permanent")

total sample number:2680
water sample number:326


In [9]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
permanent_water_layer = imagecollection.sum()
Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

## Maryland
# Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').gt(74).select('b1').rename('water_class')
# wuhan_Maryland = ee.ImageCollection([Maryland_2018,basemap1]).sum()
# Map.addLayer(wuhan_Maryland,{'min':0,"max":1,'palette':['white','red']},'wuhan_Maryland')

## JRC
# visualization = {
#     'bands': ['waterClass'],
#     'min': 0.0,
#   'max': 3.0,
#   'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
# }
# JRC = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi)
# Map.addLayer(JRC,visualization,'JRC')

In [7]:
# 添加JRC数据，做对比
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi)
JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
basemap = ee.Image.constant(0).clip(roi).rename('jrcwaterclass')
JRC_PermanentWater = ee.ImageCollection([JRC_permanentwater,basemap]).sum()
Map.addLayer(JRC_PermanentWater,{'min':0,'max':1,'palette':['white','cyan']},'jrcpermanent')

# 验证

## 使用JRC验证

In [13]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
# .select('frequency').rename('waterclass')
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

# 使用JRC做验证
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi)
JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
basemap = ee.Image.constant(0).clip(roi).rename('jrcwaterclass')
JRC_PermanentWater = ee.ImageCollection([JRC_permanentwater,basemap]).sum()
JRC_samplepoints = JRC_PermanentWater.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))
# print(JRC_samplepoints.first().getInfo())
waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))
# Overlay the points on the imagery to get training.
JRC_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

JRC total number of sample point:787
the number of water sample point:280
{'type': 'Feature', 'geometry': None, 'id': '2_0', 'properties': {'jrcwaterclass': 0, 'waterclass': 0}}
[[420, 87], [6, 274]]
total caccuracy:0.8818297331639136
kappa:0.7578918119659815


In [ ]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
# .select('frequency').rename('waterclass')
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

# 使用Maryland做验证

Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').gt(74).select('b1').rename('water_class')
wuhan_Maryland = ee.ImageCollection([Maryland_2018,basemap1]).sum()
Maryland_samplepoints =wuhan_Maryland .sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))
# print(Maryland_samplepoints.first().getInfo())
waterclass = Maryland_samplepoints.filter(ee.Filter.eq('water_class',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))
# Overlay the points on the imagery to get training.
Maryland_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['water_class'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('water_class', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

# 水体面积

In [ ]:
# calculate the area of permanent water
waterarea = wuhan_permanentwater.multiply(ee.Image.pixelArea()).divide(1e6)
areas = waterarea.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': wuhan.geometry(),
    'scale': 30,
    'maxPixels': 1e14
})
print(areas.getInfo())

In [ ]:
# from matplotlib import pyplot as plt
# import numpy as np
# import matplotlib
# from geemap import cartoee

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'bands': ['waterclass'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# # use cartoee to get a map
# ax = cartoee.get_map(wuhan_permanentwater, region=region, vis_params=vis)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'permanent surface water in 2018(MNDWI>NDVI)', fontsize=20)